Connecting to Google Drive 

In [ ]:
def connect_to_drive():
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

!pip install import_ipynb 
!pip install torchmetrics
import import_ipynb 
connect_to_drive()
%cd '/content/gdrive/My Drive/Bioinformatics2'

Mounted at /content/gdrive
/content/gdrive/My Drive/Bioinformatics2


Importing modules

In [ ]:
import torch
from utils import *
install_requirements()
from VascularDataset import *
import albumentations as A
from torchvision import transforms
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.augmentations.transforms import *
import segmentation_models_pytorch as smp
import wandb
from torchmetrics.functional import iou as iou_torch
from tqdm import tqdm
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR
from torchvision.ops.focal_loss import sigmoid_focal_loss
from segmentation_models_pytorch.losses import FocalLoss

importing Jupyter notebook from VascularDataset.ipynb


Retrieve images paths from txt

In [ ]:
input_path_training='/content/gdrive/.shortcut-targets-by-id/1mtNVywqwmtCBefeGQrY2c6F6xBQQWq30/vascular_segmentation/Train'
input_path_test='/content/gdrive/.shortcut-targets-by-id/1mtNVywqwmtCBefeGQrY2c6F6xBQQWq30/vascular_segmentation/Test'

save_model_folder='/content/gdrive/MyDrive/Bioinformatics2/saved_models/'

txt_with_training_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/training_paths_inputs.txt'
txt_with_training_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/training_paths_targets.txt'
txt_with_validation_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_inputs.txt'
txt_with_validation_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_targets.txt'
txt_with_test_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/test_paths_inputs.txt'
txt_with_test_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/test_paths_targets.txt'

imgs_training_input_split=read_list_images_path_from_txt(txt_with_training_paths_inputs)
imgs_training_target_split=read_list_images_path_from_txt(txt_with_training_paths_targets)
imgs_validation_input_split=read_list_images_path_from_txt(txt_with_validation_paths_inputs)
imgs_validation_target_split=read_list_images_path_from_txt(txt_with_validation_paths_targets)
imgs_test_input=read_list_images_path_from_txt(txt_with_test_paths_inputs)
imgs_test_target=read_list_images_path_from_txt(txt_with_test_paths_targets)

Defining transformations

In [ ]:
import albumentations as A
from albumentations import *
affine_transform=A.Compose([Flip(), ShiftScaleRotate()])
elastic_transform=A.Compose([ElasticTransform()])
pixel_wise_transform=A.Compose([GaussNoise(), HueSaturationValue(), RandomBrightnessContrast(), RandomGamma()])
affine_elastic_pixelwise_transform=A.Compose([Flip(), ShiftScaleRotate(), ElasticTransform(), GaussNoise(), HueSaturationValue(), RandomBrightnessContrast(), RandomGamma()])
affine_elastic=A.Compose([Flip(), ShiftScaleRotate(), ElasticTransform()])
slice_length=1000
resize_transform=A.Compose([RandomCrop(slice_length, slice_length)])

Defining model, loss function, metrics, and initializing Wandb

In [ ]:
from torch.nn import BCELoss
from torch.nn import BCEWithLogitsLoss

model_name='Unet++'
batch_size=2
lr=0.1
momentum=0.9
max_n_epochs=80
encoder_name='timm-resnest101e'
dataset_type='Standard'
step_size=5
size=448
gamma=0.8
transforms=affine_elastic

model = smp.Linknet(
    encoder_name=encoder_name,       
    encoder_weights='imagenet', 
    in_channels=3,
    activation=None                 
)

model.cuda()

def dice_loss(pred,target):
    numerator = 2 * torch.sum(pred * target)
    denominator = torch.sum(pred + target)
    return 1 - (numerator + 1) / (denominator + 1)

optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = Adam(model.parameters(), lr=lr)
loss=BCEWithLogitsLoss
#loss=FocalLoss(mode='binary',alpha=0.3)
scheduler=StepLR(optimizer, step_size=step_size, gamma=gamma)


wandb.init(project="Bioinformatics_segmentation_experiments_MAnet",
           config={
              "batch_size": batch_size,
              "model": model_name,
              "learning_rate": lr,
              "encoder_name": encoder_name,
              "max_n_epochs": max_n_epochs,
              "momentum": momentum,
              "optimizer": optimizer,
              "dataset": dataset_type,
              "loss": loss,
              "metrics": iou_torch,
              "transforms": transforms,
              "scheduler": scheduler,
              "step_size":step_size,
              "gamma":gamma,
              "size": size
           })

run_id=wandb.run.id
save_model_path=save_model_folder+run_id


epoch,▁▁▂▂▄▄▅▅▇▇██
training iou,▃▄▆█▁▂
training loss,▅▅▂▁█▆
validation iou,▂▆█▁▃▁
validation loss,▁▁▁█▁▁
epoch,6
training iou,0.40769
training loss,0.59761
validation iou,0.37658
validation loss,0.6576


Defining dataset and dataloaders

In [ ]:
training_dataset=VascularDataset(imgs_training_input_split, imgs_training_target_split, mean_normalization=(0.7160, 0.4878, 0.6433), std_normalization=(0.1680, 0.2098, 0.1685), size=size, transform=transforms)
validation_dataset=VascularDataset(imgs_validation_input_split, imgs_validation_target_split, mean_normalization=(0.7160, 0.4878, 0.6433), std_normalization=(0.1680, 0.2098, 0.1685), size=size)
test_dataset=VascularDataset(imgs_test_input, imgs_test_target, mean_normalization=(0.7160, 0.4878, 0.6433), std_normalization=(0.1680, 0.2098, 0.1685), size=size)

save_model_folder='/content/gdrive/MyDrive/Bioinformatics2/saved_models/'

print(f'Training dataset length: {len(training_dataset)}')
print(f'Validation dataset length: {len(validation_dataset)}')
print(f'Test dataset length: {len(test_dataset)}')

train_dataloader=DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_dataloader=DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

Training dataset length: 160
Validation dataset length: 40
Test dataset length: 10


Training, validation and logging into Wandb

In [ ]:
from torch.nn.functional import binary_cross_entropy
from torch.nn import Sigmoid

sigmoid = Sigmoid()
criterion=BCEWithLogitsLoss(torch.Tensor([2.3]).cuda())
highest_iou=0

for epoch in range(1, max_n_epochs+1, 1):

  model.train()
  training_loss=0
  training_iou=0
  training_samples=0

  for x,y in tqdm(train_dataloader): 
    optimizer.zero_grad()
    out=model(x.cuda().float())
    #weight_rebal = torch.ones_like (y) / 70.0  +  (1.0 - 1.0 / 70.0) * y
    
    my_loss=criterion(out.cuda(), y.cuda())

    for out_i,y_i in zip(out,y.cuda()):
      out_i=sigmoid(out_i.detach())
      iou=iou_torch(out_i,y_i.int())
      training_iou+=iou
      training_samples+=1

    training_loss+=my_loss.item()
    my_loss.backward()
    optimizer.step()
    
  
  print(f'Epoch #{epoch}, training loss: {training_loss/training_samples}, training iou: {training_iou/training_samples}')

  scheduler.step()

  model.eval()
  validation_loss=0
  validation_iou=0
  validation_samples=0

  for x,y in tqdm(valid_dataloader): 
    out=model(x.cuda())
    
    my_loss=criterion(out, y.cuda().float()) 
    #myloss=sigmoid_focal_loss(out, y.cuda().long(), alpha=0.3)

    for out_i,y_i in zip(out,y.cuda()):
      out_i=sigmoid(out_i.detach())
      iou=iou_torch(out_i,y_i.int())
      validation_iou+=iou
      validation_samples+=1

    validation_loss+=my_loss.item()

  print(f'Epoch #{epoch}, validation loss: {validation_loss/validation_samples}, validation iou: {validation_iou/validation_samples}')

  if validation_iou/validation_samples>highest_iou:
    print(f'saving model into {save_model_path}')
    highest_iou=validation_iou/validation_samples
    torch.save(model,f'{save_model_path}.pth')
    last_epoch_saved=epoch

  wandb.log({"training loss": training_loss/training_samples, "validation loss": validation_loss/validation_samples, "epoch":epoch})
  wandb.log({"training iou": training_iou/training_samples, "validation iou": validation_iou/validation_samples, "epoch": epoch})

print(f'Last epoch saved: {last_epoch_saved}')
wandb.log({"last epoch saved": last_epoch_saved})

100%|██████████| 80/80 [01:32<00:00,  1.16s/it]


Epoch #1, training loss: 0.5791000860928809, training iou: 0.4374639689922333


100%|██████████| 20/20 [00:13<00:00,  1.49it/s]


Epoch #1, validation loss: 0.809775921702385, validation iou: 0.3934344947338104
saving model into /content/gdrive/MyDrive/Bioinformatics2/saved_models/1fdoono1


100%|██████████| 80/80 [01:34<00:00,  1.18s/it]


...

Epoch #79, validation loss: 0.4174026370048523, validation iou: 0.6122878789901733


100%|██████████| 80/80 [01:31<00:00,  1.14s/it]


Epoch #80, training loss: 0.3694122570979169, training iou: 0.6542041301727295


100%|██████████| 20/20 [00:13<00:00,  1.48it/s]

Epoch #80, validation loss: 0.41361178904771806, validation iou: 0.613247811794281
Last epoch saved: 71


TESTING

In [ ]:
#loading model
from torch.nn import Sigmoid
model_path='/content/gdrive/MyDrive/Bioinformatics2/saved_models/1o9z0tra.pth'
model=torch.load(model_path, map_location=torch.device('cuda'))

In [ ]:
model.eval()
test_loss=0
test_iou=0
test_samples=0
criterion=BCEWithLogitsLoss(torch.Tensor([2.3]).cuda())

sigmoid = Sigmoid()
for x,y in tqdm(test_dataloader): 
  out=model(x.cuda())
  
  my_loss=criterion(out, y.cuda().float()) 
  #myloss=sigmoid_focal_loss(out, y.cuda().long(), alpha=0.3)

  for out_i,y_i in zip(out,y.cuda()):
    out_i=sigmoid(out_i.detach())
    iou=iou_torch(out_i,y_i.int())
    test_iou+=iou
    test_samples+=1

  test_loss+=my_loss.item()

print(f'test loss: {test_loss/test_samples}, test iou: {test_iou/test_samples}')

test_loss=0
test_iou=0
test_samples=0
sigmoid = Sigmoid()
for x,y in tqdm(valid_dataloader): 
  out=model(x.cuda())
  
  my_loss=criterion(out, y.cuda().float()) 
  #myloss=sigmoid_focal_loss(out, y.cuda().long(), alpha=0.3)

  for out_i,y_i in zip(out,y.cuda()):
    out_i=sigmoid(out_i.detach())
    iou=iou_torch(out_i,y_i.int())
    test_iou+=iou
    test_samples+=1

  test_loss+=my_loss.item()

print(f'test loss: {test_loss/test_samples}, test iou: {test_iou/test_samples}')


100%|██████████| 2/2 [00:04<00:00,  2.32s/it]


test loss: 0.16080617308616638, test iou: 0.6578522324562073


 88%|████████▊ | 7/8 [00:13<00:01,  1.90s/it]


RuntimeError: ignored